<img style="float: right" src="images/surfsara.png">
<br/>
<hr style="clear: both" />

# Introduction to RDDs - Apache Spark

This notebook provides an introduction to Apache Spark RDD API using PySpark. Press Shift-Enter to execute the code. You can use code completion by using tab.

During the exercises you may want to refer to [The PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html) for more information on possible transformations and actions. We will provide links to the documentation when we introduce methods on RDDs.

## The SparkContext

The SparkContext contains all the information about the way Spark is set up. When running on a cluster, the SparkContext contains the address of the cluster and will make sure operations on RDDs will be executed there. In the cell below, we create a [`SparkContext`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext) using `local mode`. This means that Spark will run locally, not on a cluster. It will offer some form of parallelism by making use of the various cores it has available.

Note, that Spark is best used in `cluster mode` where it will run on many machines simultaneously. `Local mode` is only meant for training or testing purposes. However, Spark works quite well in local mode and can be quite powerful. In order to run locally developed code on a cluster, the only thing that needs to be changed is the `SparkContext` and paths to in- and output files.

Even when working in `local mode` it is important to think of an RDD as a data structure that is distributed over many machines on a cluster, and is not available locally. The machine that contains the `SparkContext` is called the *driver*. The SparkContext will communicate with the cluster manager to make sure that the operations on RDDs will run on the cluster in the form of *workers*. It is important to realize that the driver is a separate entity from the nodes in the cluster. You can consider the notebook as being the driver.

In [4]:
# Initialize Spark
from pyspark import SparkContext, SparkConf
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')  # Spark will use all cores (*) available
    sc = SparkContext(conf=conf)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GF66\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\GF66\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Creating an RDD

There are three ways to create an RDD: by transforming an existing one, by reading in data, or by creating an RDD based on a local data structure. We show this last option below.

A Python list containing some words is used to create an RDD by calling [`parallelize`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize), a method of `SparkContext`. This list is very small and will not benefit from the parallelism of Spark. 

We then print the number of records in the RDD, by calling the `count()` method.

In [5]:
words_list = ['Dog', 'Cat', 'Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
words_rdd = sc.parallelize(words_list)
print(words_rdd.count())

8


## Map transformation 
There are two kinds of operations on RDDs: transformations and actions. Transformations take as input an RDD and produce as output another RDD (you cannot change an existing RDD, they are immutable). Computation of transformations is deferred until an *action* is executed. An action does not return an RDD, but instead returns data to the driver (for example in the form of a Python list), or writes data to disk or a database.

This *laziness* of executing transformations allows Spark to optimize computations. Only when the user wants real output, the framework will start to compute.

One of the most used transformations is [`map`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map). This is very similar to the `Map` in MapReduce. The Spark version of `Map` is a method called `map` defined on an RDD, and takes as input a single function. This function will be applied to each element in the RDD, and Spark will put the result of the application in the output RDD.

First, we present a simple Python function that takes a single word as argument and returns the word with an 's' added to it. In the next step we will use this function in a map transformation of the `words_rdd`.

Take a look at the function definition below and execute it.

In [6]:
def make_plural(word):
    return word + 's'

# Let's see if it works

print(make_plural('cat'))

cats


Next, we want to use the `make_plural` function as input for the `map` transformation on `words_rdd`.
The action [collect()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=collect#pyspark.RDD.collect) transfers the content of the RDD to the driver. The result of `collect()` will then be available to our local environment in Python. It is not an RDD but a Python list!

Note, that a large RDD may be scattered over many machines. In such a case calling `collect()` may not be a good idea, since it can take quite some time to retrieve all RDD parts.

## Exercise 1
Dưới ô cell, hãy nhập tên của hàm mà map sẽ áp dụng cho mỗi phần tử trong RDD để có được một RDD chứa các từ ở dạng số nhiều.

In [7]:
plural_rdd = words_rdd.map(make_plural)

print(plural_rdd.collect())

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


## Using lambda functions
Chúng ta có thể đạt được cùng một chức năng bằng cách sử dụng lambda functions. Trong trường hợp này, chúng ta định nghĩa `make_plural` không sử dụng `def` như chúng ta đã làm ở trên, mà là một hàm vô danh mà chúng ta định nghĩa bên trong `map` trực tiếp. Điều này là lợi ích chính của việc sử dụng lambda functions: tất cả logic xử lý của chúng ta được hiển thị trực tiếp trong các biến đổi mà chúng ta đang áp dụng.

## Exercise 2
Provide a lambda function in the cell below, that will pluralize all elements in the RDD.

In [9]:
lambda_plural_rdd = words_rdd.map(lambda word: word + 's')

print(lambda_plural_rdd.collect())

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


## Exercise 3
Một phép biến đổi khác là [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter). Nó nhận một hàm ước lượng (một hàm được đánh giá thành `true` hoặc `false`) làm đối số, và áp dụng hàm ước lượng này cho tất cả các phần tử của RDD. Chỉ có các phần tử được đánh giá thành true bởi hàm `filter` mới được chuyển vào RDD đầu ra.

Sử dụng phương thức [filter()](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=filter#pyspark.RDD.filter) của RDD để giữ lại chỉ các từ có độ dài lớn hơn ba. Sử dụng một lambda function để viết một hàm ước lượng thực hiện điều này. Tiếp theo, sử dụng `count()` để đếm số lượng từ.

Tương tự như `collect`, `count()` là một hành động. Hãy nhớ rằng các hành động kích hoạt tính toán trong Spark. Các phép biến đổi được đánh giá theo kiểu lười biếng và tính toán của chúng bị trì hoãn cho đến khi một hành động được gọi.

Có 6 từ thoả mãn phép lọc.

In [10]:
filtered_rdd = words_rdd.filter(lambda word: len(word) > 3)

print(filtered_rdd.collect())
print(filtered_rdd.count())


['Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
6


## Exercise 4

Let's do another `map` transformation on words_rdd. For each word in word_rdd determine its length, again using a lambda function.

In [11]:
word_lengths = words_rdd.map(lambda word: len(word)).collect()
print(word_lengths)


[3, 3, 6, 4, 4, 4, 10, 4]


## FlatMap transformation
Sometimes, the result of a `map` operation is a list of elements rather than a single element. Consider the following example, where we have a list of sentences, and we split each sentence:

In [12]:
sentences = sc.parallelize([
    'this is a sentence',
    'and this is another one'
])
sentences_rdd = sentences.map(str.split)
sentences_rdd.collect()


[['this', 'is', 'a', 'sentence'], ['and', 'this', 'is', 'another', 'one']]

Each element in the RDD returned by `map` is a list of words. Consequently, the result of `collect` is a list of lists, each list containing the sentences' words. Hence, a `count` of this RDD will return two:

In [13]:
sentences_rdd.count()

2

If we want to count the number of words instead, or work directly with the words, we will need to _flatten_ the list of lists into a single list. To do so, we will substitute `flatMap` for `map`. Like `map`, `flatMap` will apply the supplied function to each element in the RDD. In addition to `map`, though, it will _flatten_ the result of the operation such that a list of lists becomes a list:

In [14]:
sentences.flatMap(str.split).collect()

['this', 'is', 'a', 'sentence', 'and', 'this', 'is', 'another', 'one']

## Pair RDDs
Pair RDDs are very important within the Spark RDD API. Each element of a Pair RDD is a pair (or tuple) `(x,y)` where `x` is interpreted as being the key and `y` as the value. Spark offers quite a number of `...byKey` and `...byValues` methods that operate on pair RDDs. As we will see, these methods can be used to define functions per key, very similar to Hadoop's MapReduce.

Keys can be of any *hashable* type, which means all primitive types (numbers, strings, etc.), tuples, **but not lists or dictionaries**. Values can be of any type.

Below we define a Python string variable called `sonnet`. It is assigned Shakespeare's first sonnet in the form of a single line of text. The character `\` is used to let Python ignore the new line character. 

Execute the cell, otherwise the variable is not declared and assigned a value.

In [18]:
sonnet = "From fairest creatures we desire increase, \
That thereby beauty\'s rose might never die, \
But as the riper should by time decease, \
His tender heir might bear his memory: \
But thou contracted to thine own bright eyes, \
Feed'st thy light's flame with self-substantial fuel, \
Making a famine where abundance lies, \
Thy self thy foe, to thy sweet self too cruel: \
Thou that art now the world's fresh ornament, \
And only herald to the gaudy spring, \
Within thine own bud buriest thy content, \
And, tender churl, mak'st waste in niggarding: \
Pity the world, or else this glutton be, \
To eat the world\'s due, by the grave and thee."

## Python magic

Trong đoạn văn này, chúng ta trước tiên loại bỏ dấu câu. Ô cell tiếp theo chỉ là mã Python. Bạn có thể bỏ qua nếu tập trung của bạn chỉ vào Spark, nhưng đừng quên thực thi ô cell này.

`maketrans()` là một phương thức của Python trên chuỗi có thể thực hiện việc thay thế ký tự một cách hiệu quả. Bên dưới, chúng ta sử dụng nó để loại bỏ tất cả các ký tự dấu câu. Dấu ngoặc nhọn chỉ ra một từ điển, và biểu thức bên trong nó được gọi là một comprehension. Kết quả là một từ điển chứa các cặp khóa-giá trị, được gọi là bảng, trong đó khóa là ký tự dấu câu và giá trị là `None`. Khi thực hiện các thay thế bằng cách sử dụng `translate`, bảng này sau đó loại bỏ tất cả các mục có giá trị `None`.

In [1]:
import string

# The following line creates a translation table
table = str.maketrans({key: None for key in string.punctuation})

# Do a sample translation
s = "string. With, Punctuation?"
print(s.translate(table))


string With Punctuation


## Parallelizing the text

Trong ô cell tiếp theo, có rất nhiều việc diễn ra trong một dòng. Trước hết, văn bản ở trên được dịch - điều này có nghĩa là mỗi ký tự dấu câu được loại bỏ. Sau đó, trên kết quả này, phương thức `lower()` được áp dụng. (Đây là một phương thức của Python trên chuỗi.) Điều này chuyển chuỗi thành chữ thường. Sau đó, kết quả này được `split()`, có nghĩa là văn bản được chia thành các từ riêng lẻ. (Cũng là một phương thức của Python trên chuỗi.) Điều này dẫn đến một danh sách các từ, tất cả đều viết thường, không có dấu câu. Đây là đầu vào cho phương thức `parallelize()`, biến nó thành một RDD.

*Gọi các phương thức liên tiếp bằng cách sử dụng ký hiệu dấu chấm được gọi là chuỗi. Tất nhiên, bạn có thể thực hiện các bước này một cách riêng lẻ, nhưng việc chuỗi có thể rất tiện lợi, đặc biệt là trong Spark. Hãy xem xét từng bước riêng lẻ: trước hết là tạo RDD từ văn bản, sau đó áp dụng map cho RDD kết quả để loại bỏ dấu câu, sau đó áp dụng map cho RDD kết quả để chuyển văn bản thành chữ thường và sau đó áp dụng map cho RDD kết quả của bước đó để chia dữ liệu... Thay vì làm điều này bằng cách gọi các phương thức riêng lẻ, việc chuỗi các phương thức này tiết kiệm rất nhiều công việc gõ.*

Để chỉ hiển thị 5 phần tử đầu tiên, chúng ta sử dụng hành động [`take()`](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) của Spark. Điều này giới hạn lượng dữ liệu được gửi đến driver.

In [19]:
text_rdd = sc.parallelize(sonnet.translate(table).lower().split())
text_rdd.take(5)

['from', 'fairest', 'creatures', 'we', 'desire']

## Exercise 5

Nếu chúng ta không chia văn bản thành các từ mà biến đổi nó trực tiếp thành một RDD và bỏ qua bước `translate` và `lower`, thì điều gì sẽ xảy ra? Hãy thử điều này trong ô cell tiếp theo.

Hãy cố gắng dự đoán điều gì sẽ xảy ra. Hãy nhớ rằng một chuỗi trong Python rất tương tự một danh sách.

(Đối với một danh sách có tên là `mylist`, phần tử đầu tiên được truy cập bằng `mylist[0]`. Tương tự, `mystring[0]` sẽ trả về ký tự đầu tiên của chuỗi `mystring`.)

In [20]:
another_rdd = sc.parallelize([sonnet])
another_rdd.collect()


["From fairest creatures we desire increase, That thereby beauty's rose might never die, But as the riper should by time decease, His tender heir might bear his memory: But thou contracted to thine own bright eyes, Feed'st thy light's flame with self-substantial fuel, Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: Thou that art now the world's fresh ornament, And only herald to the gaudy spring, Within thine own bud buriest thy content, And, tender churl, mak'st waste in niggarding: Pity the world, or else this glutton be, To eat the world's due, by the grave and thee."]

## Exercise 6
We are going to count the words in `text_rdd`. As a first step, transform every word in `text_rdd` into a tuple `(<word>, 1)`. Use a lambda function.

In [21]:
pair_rdd = text_rdd.map(lambda word: (word, 1))
pair_rdd.take(5)


[('from', 1), ('fairest', 1), ('creatures', 1), ('we', 1), ('desire', 1)]

## Exercise 7

Có một hành động gọi là [countByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByKey) thực hiện việc đếm và trả về kết quả dưới dạng một từ điển Python. Sử dụng nó bên dưới để xem số lần xuất hiện của từng từ.

In [22]:
word_counts = pair_rdd.countByKey()

# Sort the word_counts dictionary by count in descending order
sorted_word_counts = sorted(word_counts.items(), key=lambda x: -x[1])

# Print the top 10 words by count
for word, count in sorted_word_counts[:10]:
    print(word, count)


the 6
thy 5
to 4
and 3
that 2
might 2
but 2
by 2
his 2
tender 2


##  reduceByKey
`countByKey` là _một hành động_, trả về số lần xuất hiện của từng từ dưới dạng một từ điển thay vì một RDD. Khi sử dụng `countByKey` với một số lượng lớn các đếm, từ điển được trả về driver có thể không vừa trong bộ nhớ.

Nếu chúng ta muốn đếm từ và giữ kết quả trong một RDD, chúng ta cần sử dụng *phép biến đổi* [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey).

Phép biến đổi này hoạt động gần giống như phép Reducer trong MapReduce của Hadoop. Nó mong đợi RDD chứa các cặp khóa-giá trị và nó sẽ thực hiện một phép reduce theo từng khóa.

Dưới dạng đầu vào, [reduceByKey](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey) nhận một hàm hai đối số sẽ được áp dụng trên các giá trị khi chúng được *nhóm theo khóa*.

## Exercise 8
Create a lambda function that does the counting and forms the input for `reduceByKey`.

In [23]:
# Note that reduceByKey takes in a function that accepts two values and returns a single value
# The function that is input to reduceByKey only works on the values. Spark will execute this function per key
count_reducer = lambda x, y: x + y
word_counts = pair_rdd.reduceByKey(count_reducer)
print(word_counts.collect())

[('but', 2), ('fuel', 1), ('where', 1), ('self', 2), ('might', 2), ('by', 2), ('lies', 1), ('else', 1), ('glutton', 1), ('we', 1), ('die', 1), ('bear', 1), ('thou', 2), ('thy', 5), ('famine', 1), ('sweet', 1), ('this', 1), ('foe', 1), ('gaudy', 1), ('buriest', 1), ('rose', 1), ('making', 1), ('only', 1), ('in', 1), ('the', 6), ('memory', 1), ('waste', 1), ('be', 1), ('fairest', 1), ('increase', 1), ('riper', 1), ('his', 2), ('abundance', 1), ('thereby', 1), ('tender', 2), ('selfsubstantial', 1), ('now', 1), ('desire', 1), ('that', 2), ('contracted', 1), ('a', 1), ('fresh', 1), ('spring', 1), ('content', 1), ('niggarding', 1), ('thee', 1), ('creatures', 1), ('beautys', 1), ('as', 1), ('heir', 1), ('thine', 2), ('own', 2), ('bright', 1), ('feedst', 1), ('lights', 1), ('too', 1), ('cruel', 1), ('within', 1), ('bud', 1), ('eat', 1), ('eyes', 1), ('ornament', 1), ('herald', 1), ('churl', 1), ('from', 1), ('should', 1), ('time', 1), ('decease', 1), ('to', 4), ('art', 1), ('and', 3), ('makst'

Instead of using `collect` we can use [takeOrdered](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered) to see the most frequent words first.

Below we show 10 elements from the RDD. The elements are pairs and we sort them by the second element (denoted by `x[1]` in the lambda function. The minus indicates descending order.

In [24]:
word_counts.takeOrdered(10, lambda x: -x[1])

[('the', 6),
 ('thy', 5),
 ('to', 4),
 ('and', 3),
 ('but', 2),
 ('self', 2),
 ('might', 2),
 ('by', 2),
 ('thou', 2),
 ('his', 2)]

## Analysing tweets
So far we have created our RDDs with our own strings and lists. Typically, though, you will read data from file or a database.

In the remainder of the notebook, we will analyse Dutch tweets that we load from file using [sc.textFile](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=textfile#pyspark.SparkContext.textFile). Each tweet is on a single line in the file, formatted a JSON dictionary. `sc.textFile` will read the each line as text, and we will need to convert this text to JSON later. We'll do this in exercise 10. The result of `sc.textFile` will be an RDD of strings, each string containing a single line (a tweet) of the file.

The file we load the tweets from is a local file in our case. Often when using Spark files reside on a distributed file system like HDFS. When creating the RDD Spark may distribute the data over many machines.

First, let's look at the first line of the data file we are going to use. We use a simple Unix command here (no Python) to view the first line of a file that resides on local disk. Notice, that this is a single tweet in JSON. 

In [31]:
# Unix bash command called head.
# The ! announces a Unix command is coming to Jupyter

!head -1 ./data/tweets.json

'head' is not recognized as an internal or external command,
operable program or batch file.


## Exercise 9
Below the call to `sc.textFile` is made. There are also empty lines in the text file (i.e. their length is equal to 0). Provide a lambda function to the subsequent `filter` call to remove these empty lines.

Lastly, print out the first tweet in the RDD by making use of the [take](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=take#pyspark.RDD.take) action.

In [ ]:
tweets_rdd = sc \
    .textFile('../data/tweets.json') \
    .filter(lambda line: len(line) > 0)
     
print(tweets_rdd.take(1))


## Conversion to JSON
Next, we are going to convert the tweets into dictionaries. For this purpose we import the Python `json` library. In Python a string `s` is converted to a dictionary by calling `json.loads(s)`.

After conversion, each tweet will be a dictionary where each key-value pair is an attribute of the tweet. Some attributes have sub-attributes, such as the ones contained under the `user` key.

## Exercise 10
Transform each tweet in the `tweets_rdd` to dictionary, and print the first tweet.

In [ ]:
import json

json_tweets_rdd = tweets_rdd.map(lambda tweet: json.loads(tweet))

# Print out the first element (tweet) of the resulting RDD
parsed_tweet = json_tweets_rdd.take(1)[0]
print(json.dumps(parsed_tweet, indent=2, sort_keys=True))


## Accessing fields in tweets
In the cell below some fields from the tweets are selected. Notice that the input `x` for the lambda function is a dictionary containing the tweet. The result of the lambda function (defined after `:`) is a list with values of the selected fields from the tweet.

You should be able to figure out how to select information from a tweet, after looking at this example.

In [ ]:
json_tweets_rdd_text = json_tweets_rdd.map(
    lambda x: [
        x['lang'],
        x['entities']['hashtags'],
        x['user']['name'],
        x['user']['screen_name'],
        x['user']['followers_count'],
        x['user']['description']
    ]
)
json_tweets_rdd_text.take(1)

## Selecting Text
We will work with the text of the tweets in the next few cells.

## Exercise 11
From the `json_tweets_rdd` select **only** the tweet text. (Do not put the text in a list, like we did above with the fields we selected there).

In [ ]:
tweet_text_rdd = json_tweets_rdd.map(lambda x: x['text'])
tweet_text_rdd.take(1)


## TweetTokenizer
The advantage of the RDD API is that it works well with unstructured data like text. We can use any function to transform RDDs. For example, to split text into words or tokens (tokenization) is often more difficult than just calling `split()` on a string. A complicating factor in tweets is the fact that they contain special characters (`#`, `@`) that have a specific meaning. Hence, tokenizing a tweet will be different from, for example, tokenizing a newspaper article.

We can make use of the Python [NLTK](http://www.nltk.org/api/nltk.tokenize.html) library for tokenization of tweets. The NLTK (Natural Language Tool Kit) contains a `TweetTokenizer` that is specially build to tokenize tweets.

In the cell below we show a short example in Python code (no RDDs yet!). A `TweetTokenizer` is created and used to tokenize an example tweet text in the variable `s`. The result is a list of tokens. Try it.

In [ ]:
from nltk.tokenize import TweetTokenizer

# Create a TweetTokenizer
tweet_tokenizer = TweetTokenizer()

# Example tweet text
s = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"

# Tokenize the tweet text
tokens = tweet_tokenizer.tokenize(s)

print(tokens)


## Exercise 12

Use `tweet_tokenizer` to tokenize all tweets in the RDD. Print the first 10 tokens, using `take`.

**Note**: we want `take` to return the first 10 tokens, not the first 10 *lists* of tokens. When you get lists you have to flatten them...

In [ ]:
# Tokenize all the tweets using tweet_tokenizer
tokenized_tweets_rdd = tweet_text_rdd.map(lambda tweet: tweet_tokenizer.tokenize(tweet))

# Flatten the list of lists of tokens
flattened_tokens_rdd = tokenized_tweets_rdd.flatMap(lambda tokens: tokens)

# Take the first 10 tokens
first_10_tokens = flattened_tokens_rdd.take(10)
print(first_10_tokens)


## Exercise 13
Count the tokens in the tweets, but only those with a length larger than 2. Show the top 15 in descending frequency.

In [ ]:
# Filter tokens with a length larger than 2
filtered_tokens_rdd = flattened_tokens_rdd.filter(lambda token: len(token) > 2)

# Count the tokens
token_counts = filtered_tokens_rdd.countByValue()

# Sort the token counts by frequency in descending order
sorted_token_counts = sorted(token_counts.items(), key=lambda x: -x[1])

# Take the top 15 tokens
top_15_tokens = sorted_token_counts[:15]

for token, count in top_15_tokens:
    print(token, count)
